# Enhancing Whisper transcriptions: pre- & post-processing techniques

This notebook offers a guide to improve the Whisper's transcriptions. We'll streamline your audio data via trimming and segmentation, enhancing Whisper's transcription quality. After transcriptions, we'll refine the output by adding punctuation, adjusting product terminology (e.g., 'five two nine' to '529'), and mitigating Unicode issues. These strategies will help improve the clarity of your transcriptions, but remember, customization based on your unique use-case may be beneficial.

## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, *-*"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8

In [2]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

In [3]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, *-*"

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23509.5

In [4]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var deployment = await Kernel.GetInputAsync("Provide deployment name");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [4]:
using Azure;
using Azure.AI.OpenAI;

In [6]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

## Setup
To get started let's import a few different libraries:

 - [Naudio](https://github.com/naudio/NAudio) is a simple and easy-to-use library for audio processing tasks such as slicing, concatenating, and exporting audio files.

 - For our audio file, we'll use a fictional earnings call written by ChatGPT and read aloud by the author.This audio file is relatively short, but hopefully provides you with an illustrative idea of how these pre and post processing steps can be applied to any audio file.

In [10]:
using System.Net.Http;
using System.IO;

// set download paths
var earningsCallUrl = "https://cdn.openai.com/API/examples/data/EarningsCall.wav";

//set local save locations
var earningsCallFilepath = "./EarningsCall.wav";

// download the file
var httpClient = new HttpClient();
using (var stream = await httpClient.GetStreamAsync(earningsCallUrl))
{
    using (var fileStream = new FileStream(earningsCallFilepath, FileMode.CreateNew))
    {
        await stream.CopyToAsync(fileStream);
    }
}

In [1]:
#r "nuget: NAudio, 2.2.1"

Installed Packages NAudio, 2.2.1

In [38]:
using NAudio.Wave;
using System.IO;

public record Silence(long Start, long End, TimeSpan Duration);


public bool IsSilence(float amplitude, sbyte threshold)
    {
        double dB = 20 * Math.Log10(Math.Abs(amplitude));
        return dB < threshold;
    }

// Find silcence in the file so we can trim it and split by silences
public Silence[] FindSilences(string fileName){
    var silences = new List<Silence>();
    using (var reader = new AudioFileReader(fileName))
    {
        var buffer = new float[reader.WaveFormat.SampleRate * 4];
    
        long start = 0;
        bool eof = false;
        sbyte silenceThreshold = -40;
        long counter = 0;
        bool detected = false;
        while (!eof)
        {
            int samplesRead = reader.Read(buffer, 0, buffer.Length);
            if (samplesRead == 0)
                {
                    eof = true;
                    if (detected){
                        double silenceSamples = (double)counter / reader.WaveFormat.Channels;
                        double silenceDuration = (silenceSamples / reader.WaveFormat.SampleRate) * 1000;
                        silences.Add(new Silence(start, counter, TimeSpan.FromMilliseconds(silenceDuration)));
                    }
                }

            for (int n = 0; n < samplesRead; n++)
            {
                if (IsSilence(buffer[n], silenceThreshold))
                {
                    detected = true;
                    counter++;
                }
                else{
                    if(detected){
                        double silenceSamples = (double)counter / reader.WaveFormat.Channels;
                        double silenceDuration = (silenceSamples / reader.WaveFormat.SampleRate) * 1000;
                        var last =silences.Count - 1;
                        if (last >= 0){
                            var gap = start - silences[last].End;
                            var gapDuration = (double)gap / reader.WaveFormat.SampleRate * 1000;
                            if (gapDuration < 500){
                              silenceDuration = silenceDuration + silences[last].Duration.TotalMilliseconds;
                             silences[last] = new Silence(silences[last].Start, counter + silences[last].End, TimeSpan.FromMilliseconds(silenceDuration));
                          }
                          else{
                           silences.Add(new Silence(start, counter, TimeSpan.FromMilliseconds(silenceDuration)));
                          }
                      }
                      else{
                            silences.Add(new Silence(start, counter, TimeSpan.FromMilliseconds(silenceDuration)));
                      }

                        start = start + counter;
                        counter = 0;
                        detected = false;
                    }
                }            
            }        
        }
    }
    return silences.ToArray();
}

In [39]:
var silences = FindSilences(earningsCallFilepath);
silences.Display();

index value 0 Silence { Start = 0, End = 3268080, Duration = 00:02:16.1572809 } Start 0 End 3268080 Duration 00:02:16.1572809 1 Silence { Start = 3268080, End = 27949, Duration = 00:00:01.1645416 } Start 3268080 End 27949 Duration 00:00:01.1645416